# Python @property
**Ref**: [Programiz](https://www.programiz.com/python-programming/property)

Python has a great concept called property which makes the life of an object oriented program much simpler. Let us build on an intuition as to why this is needed in the first place.

## Example

Let us assume that you decide to make a class that could store the temperature in degree Celsius. It would also implement a method to convert the temperature into degree Fahrenheit. One way of doing this is as follows:

In [ ]:
class Celcius():

    def __init__(self, temperature=0):
        self.temperature = temperature
    
    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32


We could make objects out of this class and manipulate the attribute `temperature` as we wish...

In [ ]:
# create a new object
man = Celcius()

# set the temperature
man.temperature = 37

# get the temperature
man.temperature

In [ ]:
# get degrees Fahrenheit
man.to_fahrenheit()

The extra decimal is due to floating point arithmetic error.

Whenever we assign or retrieve any attribute object like `temperature`, Python searches it in the object's `__dict__` dictionary.

In [ ]:
man.__dict__

Therefore, `man.temperature` internally becomes man.__dict['temperature'].

Now let us assume that our class got popular and is used in many programs. One day a client suggested that temperatures cannot go below -273.15 degC (absolute zero) and asked us to implement this value constraint. So, we decide to release version 1.01 (an upgrade) of our class

## Using Getters and Setters

An obvious solution to the above constraint will be to hide the attribute `temperature` (make it private) and define new getter and setter interfaces to manipulate. This can be done as follows:

In [ ]:
class Celcius():
    def __init__(self, temperature = 0):
        self.set_temperature(temperature)

    def to_fahrenheit(temperature):
        return (self.get_temperature() * 1.8) + 32

    # new update
    def get_temperature(self):
        return self._temperature

    def set_temperature(self, value):
        if value < -273.15:
            raise ValueError("Temperature below -273.15 is not possible")
        self._temperature = value

We can see above that the new methods `get_temperature()` and `set_temperature()` were defined and furthermore, `temperature` was replaced with `_temperature`. An underscore (_) at the beginning is used to denote private variables in Python.

Let us test this:

In [ ]:
c = Celcius(-277)
c.temperature

In [ ]:
c = Celcius(37)
c.get_temperature()

In [ ]:
c.set_temperature(10)
c.get_temperature()

In [ ]:
c.set_temperature(-300)

This update successfully implemented the new restriction. We are no longer allowed to set temperature below -273.15.as_integer_ratio

It should be noted that there are no 'private' variables in Python. They are simply norms to be followed. The language itself does not apply any restrictions

In [ ]:
c = Celcius()
c._temperature = -300 # note the underscore before temperature!
c.get_temperature()

But that is not of great concern. However, it is a big problem for all the clients who have implemented previous class in their programs. They have to modify their code from `obj.temperature` to `obj.get_temperature()` and all assignments like `obj.temperature = val` to `obj.set_temperature(val)`. This refactoring can cause headaches to clients with hundreds of thousands of lines of codes, as it is not backward compatible.

This is where **`property`** comes to the rescue.

## The Power of @property
The pythonic way to deal with the above problem is to use `property`. Here is how this could be achieved.

In [ ]:
class Celcius:
    def __init__(self, temperature=0):
        self.temperature = temperature
    
    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    def get_temperature(self):
        print("Getting value")
        return self._temperature

    def set_temperature(self, value):
        if value < -273.15:
            raise ValueError("Temperature below -273.15 is not possible")
        print("Setting value")
        self._temperature = value

    temperature = property(get_temperature, set_temperature)

In [ ]:
c = Celcius()

We added a `print()` function inside `get_temperature()` and `set_temperature()` to clearly observe that they are being executed.

The last line of code, mades a property object `temperature`. Simply put, property attaches some code (`get_temperature` and `set_temperature`) to the member attribute that accesses (`temperature`)

Any code that retreives the value of `temperature` will automatically call `get_temperature()` instead of a dictionary (__dict__) lookup. Similarly, any code that assigns a value to `temperature` will automatically call `set_temperature()`. This is one cool feature of Python.

We can see above that the `set_temperature()` was called even when we created an object.

> ## Can you guess why?

The reason is that when an object is created, `__init__()` method gets called. This method has the line `self.temperature = temperature`. This assignment automatically called `set_temperature()`

In [ ]:
c.temperature

Similarly, any access like `c.temperature` automatically calls `get_temperature()`. This is what property does. Here are a few more examples. 

In [ ]:
c.temperature = 37

In [ ]:
c.to_fahrenheit()

By using property, we can see that, we modified our class and implemented the value constraint without any change required to the client code. Thus our implementation was backward compatible and everybody is happy.

Finally note that, the actual temperature value is stored in the private variable `_temperature` (with an underscore). The attribute `temperature` (without underscore) is a property object which provides interface to this private variable.

## Digging deeper into Property

In Python, `property()` is a built-in function that creates and returns a property object. The signature of this function is:

property(fget=None, fset=None, fdel=None, doc=None) where, `fget` is function to get value of the attribute, `fset` is function to set value of the attribute, `fdel` is function to delete the attribute and `doc` is a string (like a comment). These function arguments are optional. So, a property object can simply be created as follows:

In [ ]:
property()

A property object has three methods, `getter()`, `setter()` and `deleter()` to specify `fget`, `fset` and `fdel` at a later point. This means the line

> temperature = property(get_temperature, set_temperature)

could have been broken down as:

In [ ]:
# make empty property
temperature = property()
# assign fget
temperature = temperature.getter(get_temperature)
# assign fset
temperature = temperature.setter(set_temperature)

These two pieces of code are equivalent.

Programmers familiar with decorators in Python can recognize that the above construct can be implemented as decorators.

We can further go on and not define names `get_temperature` and `set_temperature` as they are unnecessary and poollute the class namespace. For this, we reuse the name `temperature` while defining our getter and setter functions. This is how it can be done.

In [20]:
class Celsius:
    def __init__(self, temperature=0):
        self._temperature = temperature
        
    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32
    
    @property
    def temperature(self):
        print("Getting values")
        return self._temperature
    
    @temperature.setter
    def temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print("Setting value")
        self._temperature = value

In [21]:
c = Celsius()
c.temperature = 232
print(c.temperature)

Setting value
Getting values
232


The above implementation is both simple and recommended to make properties. You will most likely encounter these types of constructs when looking for property in Python